## Домашнее задание

**Ваша задача - предложить 3 способа побить бейзлайн на всех данных.**

**Baseline**:
* Precision -  0.13
* Recall -  0.25
* F1 -  0.16
* Jaccard -  0.09

Baseline implemented in here: https://github.com/mannefedov/compling_nlp_hse_course/blob/master/2020/Keyword_extraction.ipynb

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

In [46]:
%%capture
!pip3 install pymorphy2[fast]
!pip install rake-nltk

In [30]:
import json, os
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
import networkx as nx
from itertools import combinations
from gensim.summarization import keywords
from rake_nltk import Rake
nltk.download('stopwords')
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))
pd.set_option('display.max_colwidth', 100)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Utils

metrics

In [6]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

Normalizer

In [12]:
class FasterNormalizer():

    def __init__(self):
        self.morpho = MorphAnalyzer()
        self.cashe = {}

    def normalize(self, txt: str, tags: list = ['NOUN']) -> list:
        """
            returns lemmas of words with specified POS
        """
        words = self.tokenize(txt)

        res = []

        for w in words:
            if w and w not in stops:
                if w in self.cashe:
                    if self.cashe[w].tag.POS in tags:
                        res.append(self.cashe[w].normal_form)
                else:
                    r = self.morpho.parse(w)[0]
                    if r.tag.POS in tags:
                        res.append(r.normal_form)
                    self.cashe[w] = r
        return res

    def normalize_all(self, txt: str) -> list:
        """
            returns only lemmas
        """
        words = self.tokenize(txt)
        res = []
        for w in words:
            if w in self.cashe:
                res.append(self.cashe[w].normal_form)
            else:
                r = self.morpho.parse(w)[0]
                res.append(r.normal_form)
                self.cashe[w] = r
        return res

    def tokenize(self, txt:str) -> list:
        """
            tokenizes and removes punctuation from a string
        """
        return [word.strip(punct) for word in txt.lower().split()]

morph = FasterNormalizer()

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [3]:
%%capture
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_0.jsonlines.zip
!wget https://github.com/mannefedov/ru_kw_eval_datasets/raw/master/data/ng_1.jsonlines.zip
!unzip ng_0.jsonlines.zip 
!unzip ng_1.jsonlines.zip
PATH_TO_DATA = './'
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA) if file.endswith('jsonlines')]

Объединим файлы в один датасет.

In [4]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:5], axis=0, ignore_index=True)
data.shape

(1987, 5)

In [5]:
data.head(3)

,keywords,title,url,content,summary
0,"[школа, образовательные стандарты, литература, история, фгос]","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html,В среду состоялось отложенное заседание Совета по федеральным государственным образовательным ст...,"Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов вин..."
1,"[красота, законы]",У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрени...",О живительной пользе укорота при выборе между плохим и хорошим
2,"[юзефович, гражданская война, пепеляев, якутия]",Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html,Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» ...,Крепость из тел и призрак независимой Якутии


In [ ]:
data.keywords.head(1)

0    [школа, образовательные стандарты, литература, история, фгос]
Name: keywords, dtype: object

test metrics

In [7]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


Normilize

In [13]:
data['content_norm'] = data['content'].apply(morph.normalize)
data['title_norm'] = data['title'].apply(morph.normalize)
data['content_norm_all'] = data['content'].apply(morph.normalize_all)
data['title_norm_all'] = data['title'].apply(morph.normalize_all)

In [14]:
data['content_norm'].head(10)

0    [среда, заседание, совет, стандарт, фгос, министерство, образование, наука, рф, собрание, понеде...
1    [красота, глаз, угол, зрение, свет, темень, пустота, зрачок, слава, бог, красота, обход, организ...
2    [леонид, юзефович, книга, эпопея, барон, самодержец, пустынь, бестселлер, классика, жанр, роман,...
3    [гран-при, испания, евротур, гонка, трасса, барселона, календарь, минимум, год, автодром, старое...
4    [год, версия, убийство, есенин, чекист, сотрудник, гпу, телесериал, есенин, виновник, смерть, по...
5    [главное, государство, цель, образование, год, обучение, практика, выпускник, диплом, аккредитац...
6    [носитель, космос, вселенная, человек, язык, полёт, универсум, долгопрудный, дом, интеллект, тир...
7    [полгода, заявление, джордж, буш, завершение, война, ирак, война, американец, сентябрь, принцип,...
8    [время, философия, категория, понятие, сущность-элемент, отношение, действительность, знание, ис...
9    [требование, страна, россия, переговоры, вступлени

In [15]:
data['content_norm_all'].head(10)

0    [в, среда, состояться, отложить, заседание, совет, по, федеральный, государственный, образовател...
1    [хорошо, когда, красота, в, глаз, смотреть, жить, свободно, или, хотя, бы, занимать, широкий, уг...
2    [когда-то, леонид, юзефович, написать, книга, о, монгольский, эпопея, барон, унгерный, самодерже...
3    [гран-при, испания, открыть, евротур, формулы-1, гонка, на, трасса, близ, барселона, пока, остав...
4    [десять, год, назад, быть, популярный, версия, убийство, есенин, чекист, точнее, сотрудник, гпу,...
5    [, начать, с, главное, считать, что, поставить, государство, цель, абсолютно, правильный, мы, по...
6    [в, нынешний, бумажный, носитель, сойтись, как, всегда, почти, непреднамеренный, три, космос, вс...
7    [едва, ли, минуть, полгода, после, торжественный, заявление, джордж, буш, о, победоносный, завер...
8    [с, античный, время, философия, определять, категория, как, наиболее, общий, понятие, сущность-э...
9    [как, известно, один, из, требование, страна, с, к

## Approach 1: TFIDF limited to unigrams

Title+content
* Precision -  0.14 (+0.01)
* Recall -  0.28 (+0.03)
* F1 -  0.18 (+0.02)
* Jaccard -  0.11 (+0.02)

### A: Keys words are mostly nouns. Bigrams and trigrams add a lot of garbage, so if we just limit it to unigrams and take top 10 tokens, the results are higher (all four metrics)

* Precision -  0.14 (+0.01)
* Recall -  0.27 (+0.02)
* F1 -  0.18 (+0.02)
* Jaccard -  0.1 (+0.01)

Vectorize

In [253]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)
# use bigrams two
tfidf = TfidfVectorizer(ngram_range=(1,1), min_df=2)
#vectorize text
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
# transform texts intp vectors with coefficients from id2word
texts_vectors = tfidf.transform(data['content_norm_str'])


keywords = []

for row in range(texts_vectors.shape[0]):
    # it's a sparce matrix, so extract content
    row_data = texts_vectors.getrow(row)
    # transform into array and take top 10 tokens
    top_ids = row_data.toarray().argsort()[0,:-11:-1]
    # id 2 word
    keywords.append([id2word[w] for w in top_ids])

evaluate(data['keywords'], keywords)

Precision -  0.14
Recall -  0.27
F1 -  0.18
Jaccard -  0.1


### B (same approach): Playing with a number of extracting words

Same approach, but only top 5 tokens are taken. In general, decresing the number of tokens significantly improves precision (+0.15 if decreased to 1), but recall drops down. Other metrics rise a little, too. 

* Precision - 0.2 (+0.07)
* Recall - 0.2 (-0.05)
* F1 - 0.19 (+0.03)
* Jaccard - 0.12 (+0.03)

In [254]:
keywords = []

for row in range(texts_vectors.shape[0]):
    # it's a sparce matrix, so extract content
    row_data = texts_vectors.getrow(row)
    # transform into array and take top 10 tokens
    top_ids = row_data.toarray().argsort()[0,:-6:-1]
    # id 2 word
    keywords.append([id2word[w] for w in top_ids])

evaluate(data['keywords'], keywords)

Precision -  0.2
Recall -  0.2
F1 -  0.19
Jaccard -  0.12


### C: join title and content

Slight improvement:
* Precision -  0.14
* Recall -  0.28 (+0.03 to A)
* F1 -  0.18
* Jaccard -  0.11 (+0.01 to A)

In [262]:
data['content_norm_str'] = data['title_norm'].apply(' '.join) + data['content_norm'].apply(' '.join)
# use bigrams two
tfidf = TfidfVectorizer(ngram_range=(1,1), min_df=2)
#vectorize text
tfidf.fit(data['content_norm_str'])
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
# transform texts intp vectors with coefficients from id2word
texts_vectors = tfidf.transform(data['content_norm_str'])

keywords = []

for row in range(texts_vectors.shape[0]):
    # it's a sparce matrix, so extract content
    row_data = texts_vectors.getrow(row)
    # transform into array and take top 10 tokens
    top_ids = row_data.toarray().argsort()[0,:-11:-1]
    # id 2 word
    keywords.append([id2word[w] for w in top_ids])

evaluate(data['keywords'], keywords)

Precision -  0.14
Recall -  0.28
F1 -  0.18
Jaccard -  0.11


## Approach 2: Graphs

Testing different centrality measures from https://networkx.org/documentation/stable/reference/algorithms/centrality.html#current-flow-betweenness

Various centrality measures **do not** seem to improve the performance.

Best score (PageRank on nouns only) is the same as the baseline:
* Precision -  0.13
* Recall -  0.25
* F1 -  0.16
* Jaccard -  0.09


In [27]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, centrality_metrics = nx.pagerank, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    # менять тут 
    node2measure = dict(centrality_metrics(G)) 
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

PageRank by default, other metrics come from here - https://networkx.github.io/documentation/stable/reference/algorithms/centrality.html

In [32]:
keyword_nx = data['content_norm'].apply(
    lambda x: some_centrality_measure(x, window_size=10, topn=10))

evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


In [34]:
keyword_nx = data['content_norm_all'].apply(
    lambda x: some_centrality_measure(x, window_size=10, topn=10))

evaluate(data['keywords'], keyword_nx)

Precision -  0.06
Recall -  0.12
F1 -  0.07
Jaccard -  0.04


In [35]:
keyword_nx = data['content_norm'].apply(
    lambda x: some_centrality_measure(
        x, centrality_metrics = nx.load_centrality, window_size=10, topn=10))

evaluate(data['keywords'], keyword_nx)

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09


In [36]:
keyword_nx = data['content_norm_all'].apply(
    lambda x: some_centrality_measure(
        x, centrality_metrics = nx.load_centrality, window_size=10, topn=10))

evaluate(data['keywords'], keyword_nx)

Precision -  0.05
Recall -  0.11
F1 -  0.07
Jaccard -  0.04


In [37]:
keyword_nx = data['content_norm'].apply(
    lambda x: some_centrality_measure(
        x, centrality_metrics = nx.subgraph_centrality, window_size=10, topn=10))

evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


In [38]:
keyword_nx = data['content_norm_all'].apply(
    lambda x: some_centrality_measure(
        x, centrality_metrics = nx.subgraph_centrality, window_size=10, topn=10))

evaluate(data['keywords'], keyword_nx)

Precision -  0.06
Recall -  0.12
F1 -  0.08
Jaccard -  0.04


if only on titles

In [39]:
keyword_nx = data['title_norm'].apply(
    lambda x: some_centrality_measure(x, window_size=10, topn=10))

evaluate(data['keywords'], keyword_nx)

Precision -  0.2
Recall -  0.13
F1 -  0.14
Jaccard -  0.09


In [40]:
keyword_nx = data['title_norm_all'].apply(
    lambda x: some_centrality_measure(x, window_size=10, topn=10))

evaluate(data['keywords'], keyword_nx)

Precision -  0.11
Recall -  0.13
F1 -  0.11
Jaccard -  0.06


## Approach 3: Gensim

Gensim works significantly worse

Best score:

* Precision -  0.07 (-0.06)
* Recall -  0.11 (-0.14)
* F1 -  0.08 (-0.08)
* Jaccard -  0.04 (-0.05)

In [42]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


In [43]:
gensim_kws = data['content_norm_all'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])
evaluate(data['keywords'], gensim_kws)

Precision -  0.05
Recall -  0.09
F1 -  0.06
Jaccard -  0.03


In [44]:
gensim_kws = data['title_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])
evaluate(data['keywords'], gensim_kws)

Precision -  0.03
Recall -  0.01
F1 -  0.01
Jaccard -  0.01


In [45]:
gensim_kws = data['title_norm_all'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])
evaluate(data['keywords'], gensim_kws)

Precision -  0.03
Recall -  0.01
F1 -  0.01
Jaccard -  0.01


## Approach 4: RAKE

https://pypi.org/project/rake-nltk/

A bit better than the baseline:
* Precision -  0.14 (+0.01)
* Recall -  0.27 (+0.02)
* F1 -  0.17 + (0.01)
* Jaccard -  0.1 (0.01)

In [246]:
r = Rake(language='russian',
    stopwords=stops,
    punctuations=punct
)

In [248]:
keywords = []

for title, content in zip(data['title_norm'].tolist(), data['content_norm'].tolist()):
    r.extract_keywords_from_text(" ".join(title + content))
    tmp = [word for ph in r.get_ranked_phrases() for word in ph.split(' ')]
    c = Counter(tmp)
    keywords.append([keys for keys, items in c.most_common(10)])

evaluate(data['keywords'], keywords)

Precision -  0.14
Recall -  0.27
F1 -  0.17
Jaccard -  0.1


## Overall

In general, all these approaches refer to unsupervised. In order to test supervised, e.g. LDA, another larger dataset needs to be prepared to train and test on. The best unsupervised approach is TFIDF.